In [13]:
import numpy as np
from sklearn.linear_model import LinearRegression
import scipy
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from tqdm import tqdm
from scipy.linalg.lapack import zggev
from scipy.linalg import block_diag
import pandas as pd
from sklearn.cross_decomposition import CCA, PLSCanonical
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from direct_effect_analysis import * 
from utils import *
from mvlearn.embed import GCCA
from pgmpy.estimators import PC
from plotnine import ggplot, aes, geom_line, geom_ribbon, scale_x_log10, scale_x_continuous, labs, theme, facet_wrap, ggsave, theme_bw, element_text, facet_grid, scale_color_manual

In [2]:
algorithms = ['T_D', 'T_F', 'T_S', 'PCA', 'pCCA']
B_conds = ['d', '1', '1/d', '1/d^2']
Sigma_conds = ['d', '1', '1/d', '1/d^2']
results = {B_cond:{Sigma_cond:{algo: {} for algo in algorithms } for Sigma_cond in Sigma_conds} for B_cond in B_conds} 

In [5]:
p, r, d, q, rk = 10, 10, 100, 1, 10
N = 100
dimensions = [2, 5, 10, 20, 50, 100]

In [6]:
a, b, c = 0.1, 0.1, 0.8
B = 10
for B_cond in tqdm(B_conds):
    for Sigma_cond in Sigma_conds:
        for algo in algorithms:
            # print(noise, algo)
            CORRS = []
            for d in dimensions:
                d = int(d)
                CORR = []
                for j in range(B):

                    # Generate parameters
                    _, A_z, beta, gamma = parameters(r, p, q, d, False)

                    if B_cond == 'd':
                        A_x = np.array([i for i in range(d)])[:, None].T
                    elif B_cond == '1':
                        A_x = np.array([1 for i in range(d)])[:, None].T
                    elif B_cond == '1/d':
                        A_x = np.array([1/(i+1) for i in range(d)])[:, None].T
                    elif B_cond == '1/d^2':
                        A_x = np.array([1/((i+1)**2) for i in range(d)])[:, None].T

                    if Sigma_cond == 'd':
                        Sigma = np.diag(np.array([i for i in range(d)]))
                    elif Sigma_cond == '1':
                        Sigma = np.identity(d)
                    elif Sigma_cond == '1/d':
                        Sigma = np.diag(np.array([1/(i+1) for i in range(d)]))
                    elif Sigma_cond == '1/d^2':
                        Sigma = np.diag(np.array([1/((i+1)**2) for i in range(d)]))


                    # Generate training and test data
                    if d>50 and N < 200:
                        alpha = 1e2
                    else :
                        alpha = 1e-5
                    X_train, Y_train, Z_train, Y_x_train = generate_data_Sigma(N, p, r, d, beta, gamma, A_x, A_z, Sigma, a=a, b=b, c=c)
                    dea = DirectEffectAnalysis(type=algo, alpha=1e-5)
                    dea.fit(X_train, Y_train, Z_train)
                    Y_hat = dea.transform(X_train, Y_train, Z_train)
                    corr = np.abs(np.corrcoef(Y_hat, Y_x_train[:,0])[0, 1])
                    
                    # Compute MSE for each iteration
                    CORR.append(corr)
                CORRS.append(CORR)
            # Append MSE for current sample size
            results[B_cond][Sigma_cond][algo] = CORRS

100%|██████████| 4/4 [00:21<00:00,  5.26s/it]


In [15]:
# Assuming you have the following data structures
algo_names = {'T_D':'T_D', 'T_F':'T_F', 'T_S':'T_S', 'PCA':'PCA', 'pCCA':'pCCA'}
# results, B_conds, Sigma_conds, algorithms, dimensions are defined elsewhere

# Prepare data for ggplot
data = []
for B_cond in B_conds:
    for Sigma_cond in Sigma_conds:
        for algo in algorithms:
            algo_data = results[B_cond][Sigma_cond][algo]
            values = np.array(algo_data)
            median = np.percentile(values, 50, axis=1)
            lower = np.percentile(values, 5, axis=1)
            upper = np.percentile(values, 95, axis=1)
            
            for i, dim in enumerate(dimensions):
                data.append({
                    'Dimension': dim,
                    'Median': median[i],
                    'Lower': lower[i],
                    'Upper': upper[i],
                    'Algorithm': algo_names[algo],
                    'Sigma': Sigma_cond,
                    'B': B_cond  # Add weight name to the data
                })

# Create a DataFrame
df = pd.DataFrame(data)
df['B'] = df['B'].astype('category')
df['B'] = df['B'].cat.reorder_categories(['d', '1', '1/d', '1/d^2'])

df['Sigma'] = df['Sigma'].astype('category')
df['Sigma'] = df['Sigma'].cat.reorder_categories(['d', '1', '1/d', '1/d^2'])


# df2 = df[df['Algorithm']!='PCA']
df2 = df

# Create a new column for the facet label
df2['Facet_Label'] = '$\\mathbf{\\sigma}=' + df2['Sigma'].astype(str) + ',\\hspace{1} \\mathbf{b}_i=' + df2['B'].astype(str) + '$'

p = (ggplot(df2, aes(x='Dimension', y='Median', color='Algorithm', fill='Algorithm'))
     + geom_line(size=1.5)
     + geom_ribbon(aes(ymin='Lower', ymax='Upper', fill='Algorithm', color='Algorithm'), alpha=0.2)
     + scale_color_manual(values={'PCA': 'rgba(255, 0, 0, 0.3)', 'Alg2': 'rgba(0, 255, 0, 0.3)', 'Alg3': 'rgba(0, 0, 255, 0.3)'})  # Adjust opacity
     + scale_x_log10(breaks=dimensions)  
     + labs(x='Dimension (d)', y='Absolute Correlation')
     + facet_grid(rows="B", cols="Sigma", labeller="label_both")
     + theme_bw()
     + theme(
         legend_position='bottom', 
         figure_size=(12, 12),
         axis_title=element_text(size=23),
         axis_text=element_text(size=22),
         axis_text_x=element_text(angle=55, hjust=1),
         legend_title=element_text(size=22),
         legend_text=element_text(size=25),
         strip_text=element_text(size=18)
     )
)


# Display the plot
print(p)


/tmp/ipykernel_4552/3847085611.py:64: FutureWarning: Using print(plot) to draw and show the plot figure is deprecated and will be removed in a future version. Use plot.show().
/home/homer/Documents/Projects/Anchor_Regression_Detection/.conda/lib/python3.9/site-packages/plotnine/scales/scale_manual.py:44: PlotnineWarning: The palette of scale_color_manual can return a maximum of 3 values. 5 were requested from it.


ValueError: Invalid RGBA argument: 'rgba(255, 0, 0, 0.3)'